In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def save_obj(obj, name ):
    with open('/content/drive/My Drive/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
save_obj(word_to_index, 'word_to_index' )

In [ ]:
import numpy as np
import pickle
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words("english")) - {'no' , 'not', 'very'}

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#functions

In [ ]:
def cleaned_text_method2(text):

    cleaned_text = re.sub('http.*','',text)

    # specific
    cleaned_text = re.sub(r"won\'t", "will not", cleaned_text)
    cleaned_text = re.sub(r"can\'t", "can not", cleaned_text)

    # general
    cleaned_text = re.sub(r"n\'t", " not", cleaned_text)
    cleaned_text = re.sub(r"\'re", " are", cleaned_text)
    cleaned_text = re.sub(r"\'s", " is", cleaned_text)
    cleaned_text = re.sub(r"\'d", " would", cleaned_text)
    cleaned_text = re.sub(r"\'ll", " will", cleaned_text)
    cleaned_text = re.sub(r"\'t", " not", cleaned_text)
    cleaned_text = re.sub(r"\'ve", " have", cleaned_text)
    cleaned_text = re.sub(r"\'m", " am", cleaned_text)
    
    cleaned_text = cleaned_text.lower()

    cleaned_text = " ".join([w for w in cleaned_text.split() if not w in stops])
    
    cleaned_text = re.sub("[^a-zA-Z]", " ",cleaned_text)

    #cleaned_text = " ".join([w for w in cleaned_text.split() if len(w) > 1])

    cleaned_text = re.sub('\s+',' ',cleaned_text)
                              
    return cleaned_text 

In [ ]:
def sentences_to_indices(X, word_to_index, maxLen):

  m = X.shape[0] # number of training examples
    
  X_indices = np.zeros((m, maxLen),dtype=int)
  i = 0
  for s in X:

    sentence_words = s.split()
    
    j = 0
    
    for w in sentence_words:
      if j >= maxLen:
        break
      if w in word_to_index:
        X_indices[i, j] = int(word_to_index[w])
      j = j + 1
    i = i + 1
        
  return X_indices

In [ ]:
def create_mpdel(NB_WORDS, GLOVE_DIM, MAX_LEN, weights_file_path):
  model=Sequential()
  model.add(Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN))
  model.add(LSTM(128 , return_sequences=True))
  model.add(Dropout(0.2))
  model.add(LSTM(128 , return_sequences=False))
  model.add(Dropout(0.2))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(5, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.load_weights(weights_file_path)
  return model

In [ ]:
def load_obj(word_to_index_path):
    with open(word_to_index_path, 'rb') as f:
        return pickle.load(f)

In [ ]:
def init(word_to_index_path, weights_file_path):
  word_to_index = load_obj(word_to_index_path)

  NB_WORDS = len(word_to_index)+1
  MAX_LEN = 99
  GLOVE_DIM = 50

  model = create_mpdel(NB_WORDS, GLOVE_DIM, MAX_LEN, weights_file_path)
  return model, word_to_index, MAX_LEN

In [ ]:
def predict(x_test, model, word_to_index, MAX_LEN):
  for i in range(len(x_test)):
    x_test[i] = cleaned_text_method2(x_test[i])
  X_test_indices = sentences_to_indices(x_test, word_to_index, MAX_LEN)
  y_pred = model.predict(X_test_indices)

  y_pred[:,3] = y_pred[:,3] + 0.02
  y_pred[:,4] = y_pred[:,4] - 0.02
  y_pred_bool = np.argmax(y_pred, axis=1)
  y_pred_bool = y_pred_bool + 1

  try:
    reate1_comment, = x_test[np.where(np.isclose(y_pred[:,0] , y_pred[y_pred_bool == 1][:,0].max()))[0]]
  except:
    reate1_comment = ""
  try:
    reate2_comment, =  x_test[np.where(np.isclose(y_pred[:,1] , y_pred[y_pred_bool == 2][:,1].max()))[0]]
  except:
    reate2_comment = ""
  try:
    reate3_comment, =  x_test[np.where(np.isclose(y_pred[:,2] , y_pred[y_pred_bool == 3][:,2].max()))[0]]
  except:
    reate3_comment = ""
  try:
    reate4_comment, =  x_test[np.where(np.isclose(y_pred[:,3] , y_pred[y_pred_bool == 4][:,3].max()))[0]]
  except:
    reate4_comment = ""
  try:
    reate5_comment, =  x_test[np.where(np.isclose(y_pred[:,4] , y_pred[y_pred_bool == 5][:,4].max()))[0]]
  except:
    reate5_comment = ""

  return y_pred_bool, reate1_comment, reate2_comment, reate3_comment, reate4_comment, reate5_comment

initializing

In [ ]:
word_to_index_path = '/content/drive/My Drive/word_to_index.pkl'
weights_file_path = '/content/drive/My Drive/model3.h5'

model, word_to_index, MAX_LEN = init(word_to_index_path, weights_file_path)

prediction

In [ ]:
x_test = np.array(["You were responsive to my urgent shipping needs,Thank you.",
                   "This prouduct is too expensive and i don't like the price",
                   "The Movie was extremly good but a lot of things was not completed",
                   "I wanted to let you know that I am quite pleased with you fast delivery & your product , which works well. As advertised",
                   "As per the subject - your customer service is awesome ",
                   "your website is awesome as are you prices - sold - you have a new customer! THANKS",
                   "The Services Was not good enough after selling. Thank You",
                   "This goes to the wonderful, very helpful, and very knowledgeable team of people",
                   "Love this t-shirt it has a good price too THANKSSS",
                   "Beatuiful movie i like it very much, GOOD TEAM",
                   "The customar services is not effective please take care of the customers",
                   "the materials isn't good enough so the products broke from the second time from the begining of use",
                   "This actions involves racism so it's bad habbits",
                   "i think it is good enough to complete it",
                   "it is very good"])

predictions = predict(x_test, model, word_to_index, MAX_LEN)
print(predictions)

(array([5, 2, 3, 5, 5, 5, 1, 5, 5, 5, 1, 2, 1, 4, 4]), 'actions involves racism bad habbits', 'materials not good enough products broke second time begining use', 'movie extremly good lot things not completed', 'think good enough complete', 'website awesome prices sold new customer thanks')


#OOP

In [ ]:
class CommentRater:

  def __init__(self, word_to_index_path, weights_file_path):
    self.word_to_index = self.load_obj(word_to_index_path)
    self.NB_WORDS = len(self.word_to_index)+1
    self.MAX_LEN = 99
    self.GLOVE_DIM = 50
    self.model = self.create_mpdel(self.NB_WORDS, self.GLOVE_DIM, self.MAX_LEN, weights_file_path)



  def cleaned_text_method2(self, text):
    cleaned_text = re.sub('http.*','',text)
    # specific
    cleaned_text = re.sub(r"won\'t", "will not", cleaned_text)
    cleaned_text = re.sub(r"can\'t", "can not", cleaned_text)
    # general
    cleaned_text = re.sub(r"n\'t", " not", cleaned_text)
    cleaned_text = re.sub(r"\'re", " are", cleaned_text)
    cleaned_text = re.sub(r"\'s", " is", cleaned_text)
    cleaned_text = re.sub(r"\'d", " would", cleaned_text)
    cleaned_text = re.sub(r"\'ll", " will", cleaned_text)
    cleaned_text = re.sub(r"\'t", " not", cleaned_text)
    cleaned_text = re.sub(r"\'ve", " have", cleaned_text)
    cleaned_text = re.sub(r"\'m", " am", cleaned_text)
    
    cleaned_text = cleaned_text.lower()
    cleaned_text = " ".join([w for w in cleaned_text.split() if not w in stops])
    cleaned_text = re.sub("[^a-zA-Z]", " ",cleaned_text)
    #cleaned_text = " ".join([w for w in cleaned_text.split() if len(w) > 1])
    cleaned_text = re.sub('\s+',' ',cleaned_text)
                              
    return cleaned_text 



  def sentences_to_indices(self, X, word_to_index, maxLen):
    m = X.shape[0] # number of training examples
    X_indices = np.zeros((m, maxLen),dtype=int)
    i = 0
    for s in X:
      sentence_words = s.split()
      j = 0
      for w in sentence_words:
        if j >= maxLen:
          break
        if w in word_to_index:
          X_indices[i, j] = int(word_to_index[w])
        j = j + 1
      i = i + 1
          
    return X_indices



  def create_mpdel(self, NB_WORDS, GLOVE_DIM, MAX_LEN, weights_file_path):
    model=Sequential()
    model.add(Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN))
    model.add(LSTM(128 , return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128 , return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.load_weights(weights_file_path)
    return model



  def load_obj(self, word_to_index_path):
      with open(word_to_index_path, 'rb') as f:
          return pickle.load(f)



  def predict(self, x_test):
    for i in range(len(x_test)):
      x_test[i] = self.cleaned_text_method2(x_test[i])
    X_test_indices = self.sentences_to_indices(x_test, self.word_to_index, self.MAX_LEN)
    y_pred = self.model.predict(X_test_indices)
    y_pred[:,3] = y_pred[:,3] + 0.02
    y_pred[:,4] = y_pred[:,4] - 0.02
    y_pred_bool = np.argmax(y_pred, axis=1)
    y_pred_bool = y_pred_bool + 1

    try:
      reate1_comment, = x_test[np.where(np.isclose(y_pred[:,0] , y_pred[y_pred_bool == 1][:,0].max()))[0]]
    except:
      reate1_comment = ""
    try:
      reate2_comment, =  x_test[np.where(np.isclose(y_pred[:,1] , y_pred[y_pred_bool == 2][:,1].max()))[0]]
    except:
      reate2_comment = ""
    try:
      reate3_comment, =  x_test[np.where(np.isclose(y_pred[:,2] , y_pred[y_pred_bool == 3][:,2].max()))[0]]
    except:
      reate3_comment = ""
    try:
      reate4_comment, =  x_test[np.where(np.isclose(y_pred[:,3] , y_pred[y_pred_bool == 4][:,3].max()))[0]]
    except:
      reate4_comment = ""
    try:
      reate5_comment, =  x_test[np.where(np.isclose(y_pred[:,4] , y_pred[y_pred_bool == 5][:,4].max()))[0]]
    except:
      reate5_comment = ""

    return y_pred_bool, reate1_comment, reate2_comment, reate3_comment, reate4_comment, reate5_comment

In [ ]:
word_to_index_path = '/content/drive/My Drive/word_to_index.pkl'
weights_file_path = '/content/drive/My Drive/model3.h5'

model_rater = CommentRater(word_to_index_path, weights_file_path)

In [ ]:
comments = ["You were responsive to my urgent shipping needs,Thank you.",
                   "This prouduct is too expensive and i don't like the price",
                   "The Movie was extremly good but a lot of things was not completed",
                   "I wanted to let you know that I am quite pleased with you fast delivery & your product , which works well. As advertised",
                   "As per the subject - your customer service is awesome ",
                   "your website is awesome as are you prices - sold - you have a new customer! THANKS",
                   "The Services Was not good enough after selling. Thank You",
                   "This goes to the wonderful, very helpful, and very knowledgeable team of people",
                   "Love this t-shirt it has a good price too THANKSSS",
                   "Beatuiful movie i like it very much, GOOD TEAM",
                   "The customar services is not effective please take care of the customers",
                   "the materials isn't good enough so the products broke from the second time from the begining of use",
                   "This actions involves racism so it's bad habbits",
                   "i think it is good enough to complete it",
                   "it is very good"]
x_test = np.array(comments)
y_predict, reate1_comment, reate2_comment, reate3_comment, reate4_comment, reate5_comment = model_rater.predict(x_test)
for i in range(len(x_test)):
  print('comment : ' , comments[i])
  print('rate    : ' , y_predict[i])
  print()

comment :  You were responsive to my urgent shipping needs,Thank you.
rate    :  5

comment :  This prouduct is too expensive and i don't like the price
rate    :  2

comment :  The Movie was extremly good but a lot of things was not completed
rate    :  3

comment :  I wanted to let you know that I am quite pleased with you fast delivery & your product , which works well. As advertised
rate    :  5

comment :  As per the subject - your customer service is awesome 
rate    :  5

comment :  your website is awesome as are you prices - sold - you have a new customer! THANKS
rate    :  5

comment :  The Services Was not good enough after selling. Thank You
rate    :  1

comment :  This goes to the wonderful, very helpful, and very knowledgeable team of people
rate    :  5

comment :  Love this t-shirt it has a good price too THANKSSS
rate    :  5

comment :  Beatuiful movie i like it very much, GOOD TEAM
rate    :  5

comment :  The customar services is not effective please take care of th